# Lab 4.5.1: RAG Demo - Solutions

Complete solutions for the RAG demo exercises.

---

## Solution 1: Greeting with Time

Add current time to the greeting.

In [ ]:
import gradio as gr
from datetime import datetime

with gr.Blocks() as time_demo:
    gr.Markdown("# Greeting with Time")
    
    name_input = gr.Textbox(label="Your Name")
    greeting_output = gr.Textbox(label="Greeting")
    greet_button = gr.Button("Greet Me!")
    
    def greet_with_time(name):
        current_time = datetime.now().strftime("%H:%M")
        return f"Hello, {name}! It's currently {current_time}. Welcome!"
    
    greet_button.click(
        fn=greet_with_time,
        inputs=[name_input],
        outputs=[greeting_output]
    )

# time_demo.launch()

## Solution 2: Three-Column Layout

Create a 3-column layout with file upload, main content, and settings.

In [ ]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft()) as three_column_demo:
    gr.Markdown("# Three-Column Layout Demo")
    
    with gr.Row():
        # Left column - File upload (scale=1)
        with gr.Column(scale=1):
            gr.Markdown("### 📁 Files")
            file_upload = gr.File(
                label="Upload Documents",
                file_count="multiple"
            )
            upload_btn = gr.Button("Process Files", variant="secondary")
            file_status = gr.Textbox(label="Status", lines=3)
        
        # Middle column - Main content (scale=2)
        with gr.Column(scale=2):
            gr.Markdown("### 💬 Main Content")
            chat_input = gr.Textbox(
                label="Your Message",
                placeholder="Type here...",
                lines=3
            )
            chat_output = gr.Textbox(
                label="Response",
                lines=8
            )
            with gr.Row():
                send_btn = gr.Button("Send", variant="primary")
                clear_btn = gr.Button("Clear")
        
        # Right column - Settings (scale=1)
        with gr.Column(scale=1):
            gr.Markdown("### ⚙️ Settings")
            model_select = gr.Dropdown(
                choices=["llama3.2:3b", "llama3.1:8b", "mistral:7b"],
                value="llama3.2:3b",
                label="Model"
            )
            temperature = gr.Slider(
                minimum=0, maximum=1, value=0.7,
                label="Temperature"
            )
            max_tokens = gr.Number(
                value=512,
                label="Max Tokens"
            )
            
            with gr.Accordion("Advanced", open=False):
                top_p = gr.Slider(0, 1, 0.9, label="Top-P")
                repeat_penalty = gr.Slider(1, 2, 1.1, label="Repeat Penalty")
    
    # Event handlers
    def process_files(files):
        if not files:
            return "No files uploaded"
        return f"Processed {len(files)} files"
    
    def respond(message, model, temp):
        return f"[{model}] Response to: {message}\n(temp={temp})"
    
    upload_btn.click(process_files, [file_upload], [file_status])
    send_btn.click(respond, [chat_input, model_select, temperature], [chat_output])
    clear_btn.click(lambda: ("", ""), outputs=[chat_input, chat_output])

# three_column_demo.launch()

## Solution 3: Complete RAG Demo with All Features

Full-featured RAG demo with streaming, theme toggle, and export.

In [ ]:
import gradio as gr
import chromadb
import ollama
import json
from datetime import datetime
from pathlib import Path
from typing import List, Tuple, Dict


class EnhancedRAGBackend:
    """
    Enhanced RAG backend with additional features:
    - Streaming responses
    - Conversation export
    - Analytics tracking
    """
    
    def __init__(self):
        self.client = chromadb.Client()
        self.collection = self.client.get_or_create_collection("docs")
        self.llm_model = "llama3.2:3b"
        self.embed_model = "nomic-embed-text"
        self.n_results = 3
        self.temperature = 0.7
        
        # Analytics
        self.query_count = 0
        self.avg_response_time = 0
    
    def chunk_text(self, text: str, chunk_size: int = 500) -> List[str]:
        chunks = []
        for i in range(0, len(text), chunk_size - 50):
            chunk = text[i:i+chunk_size]
            if len(chunk) > 50:
                chunks.append(chunk.strip())
        return chunks
    
    def index_document(self, text: str, filename: str) -> Tuple[int, str]:
        try:
            chunks = self.chunk_text(text)
            if not chunks:
                return 0, "Document too short"
            
            embeddings = []
            for chunk in chunks:
                resp = ollama.embeddings(model=self.embed_model, prompt=chunk)
                embeddings.append(resp["embedding"])
            
            base_id = filename.replace(" ", "_").replace(".", "_")
            ids = [f"{base_id}_{i}" for i in range(len(chunks))]
            
            self.collection.add(
                ids=ids,
                embeddings=embeddings,
                documents=chunks,
                metadatas=[{"source": filename}] * len(chunks)
            )
            
            return len(chunks), f"Indexed {len(chunks)} chunks"
        except Exception as e:
            return 0, f"Error: {str(e)}"
    
    def search(self, query: str) -> List[Dict]:
        resp = ollama.embeddings(model=self.embed_model, prompt=query)
        results = self.collection.query(
            query_embeddings=[resp["embedding"]],
            n_results=self.n_results
        )
        
        chunks = []
        for i in range(len(results["documents"][0])):
            chunks.append({
                "text": results["documents"][0][i],
                "source": results["metadatas"][0][i]["source"],
                "distance": results["distances"][0][i] if "distances" in results else 0
            })
        return chunks
    
    def chat_stream(self, query: str, history: List[Tuple[str, str]]):
        """Streaming chat response."""
        import time
        start_time = time.time()
        
        context_chunks = self.search(query)
        if not context_chunks:
            yield "No documents indexed. Please upload some first!"
            return
        
        context = "\n\n".join([c["text"] for c in context_chunks])
        
        system = """Answer based on the context. Cite sources."""
        
        messages = [{"role": "system", "content": system}]
        for u, a in history[-5:]:
            messages.append({"role": "user", "content": u})
            messages.append({"role": "assistant", "content": a})
        
        messages.append({"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"})
        
        response_text = ""
        for chunk in ollama.chat(
            model=self.llm_model,
            messages=messages,
            stream=True,
            options={"temperature": self.temperature}
        ):
            token = chunk["message"]["content"]
            response_text += token
            yield response_text
        
        # Update analytics
        elapsed = time.time() - start_time
        self.query_count += 1
        self.avg_response_time = (
            (self.avg_response_time * (self.query_count - 1) + elapsed) / 
            self.query_count
        )
    
    def get_sources_markdown(self, query: str) -> str:
        chunks = self.search(query)
        sources_md = "**Sources:**\n"
        for i, chunk in enumerate(chunks, 1):
            confidence = 1 - chunk["distance"]
            sources_md += f"\n{i}. **{chunk['source']}** ({confidence:.0%})\n"
            sources_md += f"   > {chunk['text'][:100]}...\n"
        return sources_md
    
    def get_stats(self) -> Dict:
        return {
            "total_chunks": self.collection.count(),
            "queries_processed": self.query_count,
            "avg_response_time": f"{self.avg_response_time:.2f}s",
            "model": self.llm_model
        }
    
    def export_history(self, history: List[Tuple[str, str]]) -> str:
        """Export chat history as JSON."""
        export_data = {
            "exported_at": datetime.now().isoformat(),
            "model": self.llm_model,
            "messages": [
                {"user": u, "assistant": a}
                for u, a in history
            ]
        }
        return json.dumps(export_data, indent=2)


def create_enhanced_rag_demo():
    """Create an enhanced RAG demo with streaming and export."""
    
    rag = EnhancedRAGBackend()
    
    # Theme options
    light_theme = gr.themes.Soft()
    dark_theme = gr.themes.Default()
    
    with gr.Blocks(theme=light_theme, title="Enhanced RAG Demo") as demo:
        gr.Markdown("""
        # 🚀 Enhanced RAG Demo
        
        Features: Streaming responses, conversation export, analytics
        """)
        
        with gr.Tabs():
            # Documents Tab
            with gr.TabItem("📁 Documents"):
                with gr.Row():
                    with gr.Column(scale=2):
                        files = gr.File(
                            label="Upload Documents",
                            file_count="multiple",
                            file_types=[".txt", ".md", ".pdf"]
                        )
                        with gr.Row():
                            index_btn = gr.Button("📥 Index", variant="primary")
                            clear_btn = gr.Button("🗑️ Clear")
                        status = gr.Textbox(label="Status", lines=5)
                    
                    with gr.Column(scale=1):
                        doc_count = gr.Number(label="Chunks Indexed", value=0)
                        stats_display = gr.JSON(label="Statistics")
                        refresh_stats_btn = gr.Button("🔄 Refresh Stats")
            
            # Chat Tab with Streaming
            with gr.TabItem("💬 Chat"):
                with gr.Row():
                    with gr.Column(scale=3):
                        chatbot = gr.Chatbot(height=450, show_copy_button=True)
                        
                        with gr.Row():
                            msg = gr.Textbox(
                                label="Message",
                                placeholder="Ask about your documents...",
                                scale=4,
                                show_label=False
                            )
                            send = gr.Button("Send", variant="primary", scale=1)
                        
                        with gr.Row():
                            clear_chat = gr.Button("🗑️ Clear")
                            export_btn = gr.Button("📤 Export")
                            export_file = gr.File(label="Download", visible=False)
                    
                    with gr.Column(scale=1):
                        gr.Markdown("### 📚 Sources")
                        sources = gr.Markdown("*Ask a question to see sources*")
            
            # Settings Tab
            with gr.TabItem("⚙️ Settings"):
                with gr.Row():
                    with gr.Column():
                        model_select = gr.Dropdown(
                            choices=["llama3.2:3b", "llama3.1:8b"],
                            value="llama3.2:3b",
                            label="LLM Model"
                        )
                        chunks_slider = gr.Slider(1, 10, 3, step=1, label="Chunks")
                        temp_slider = gr.Slider(0, 1, 0.7, step=0.1, label="Temperature")
                        save_btn = gr.Button("💾 Save Settings", variant="primary")
                        settings_status = gr.Textbox(label="Status")
        
        # Event Handlers
        def process_files(files):
            if not files:
                return "No files", 0
            results = []
            for f in files:
                with open(f.name, 'r', errors='ignore') as file:
                    text = file.read()
                n, msg = rag.index_document(text, Path(f.name).name)
                results.append(f"✅ {Path(f.name).name}: {msg}")
            return "\n".join(results), rag.collection.count()
        
        def clear_docs():
            rag.client.delete_collection("docs")
            rag.collection = rag.client.create_collection("docs")
            return "Cleared!", 0
        
        def chat_stream(message, history):
            if not message:
                return history, ""
            
            history = history + [[message, ""]]
            
            for response in rag.chat_stream(message, history[:-1]):
                history[-1][1] = response
                yield history, rag.get_sources_markdown(message)
        
        def export_chat(history):
            json_content = rag.export_history(history)
            filename = f"chat_export_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            with open(filename, 'w') as f:
                f.write(json_content)
            return gr.update(visible=True, value=filename)
        
        def save_settings(model, chunks, temp):
            rag.llm_model = model
            rag.n_results = int(chunks)
            rag.temperature = temp
            return f"Saved: model={model}, chunks={chunks}, temp={temp}"
        
        # Wire events
        index_btn.click(process_files, [files], [status, doc_count])
        clear_btn.click(clear_docs, outputs=[status, doc_count])
        refresh_stats_btn.click(lambda: rag.get_stats(), outputs=[stats_display])
        
        send.click(chat_stream, [msg, chatbot], [chatbot, sources])
        msg.submit(chat_stream, [msg, chatbot], [chatbot, sources])
        clear_chat.click(lambda: ([], "*Ask a question*"), outputs=[chatbot, sources])
        export_btn.click(export_chat, [chatbot], [export_file])
        
        save_btn.click(save_settings, [model_select, chunks_slider, temp_slider], [settings_status])
    
    return demo


# Create and test
print("Enhanced RAG Demo created!")
# enhanced_demo = create_enhanced_rag_demo()
# enhanced_demo.launch()

## Solution 4: Challenge - Multi-language Support

Add language selector and translate UI elements.

In [ ]:
import gradio as gr

# Translation dictionary
TRANSLATIONS = {
    "en": {
        "title": "RAG Chat Demo",
        "upload": "Upload Documents",
        "index": "Index Documents",
        "clear": "Clear",
        "message": "Type your message...",
        "send": "Send",
        "settings": "Settings",
        "model": "Model",
        "temperature": "Temperature"
    },
    "es": {
        "title": "Demo de Chat RAG",
        "upload": "Subir Documentos",
        "index": "Indexar Documentos",
        "clear": "Limpiar",
        "message": "Escribe tu mensaje...",
        "send": "Enviar",
        "settings": "Configuración",
        "model": "Modelo",
        "temperature": "Temperatura"
    },
    "fr": {
        "title": "Démo Chat RAG",
        "upload": "Télécharger des Documents",
        "index": "Indexer les Documents",
        "clear": "Effacer",
        "message": "Tapez votre message...",
        "send": "Envoyer",
        "settings": "Paramètres",
        "model": "Modèle",
        "temperature": "Température"
    }
}

def create_multilang_demo():
    """Create a multi-language demo."""
    
    with gr.Blocks() as demo:
        # Language selector at top
        with gr.Row():
            lang = gr.Dropdown(
                choices=["en", "es", "fr"],
                value="en",
                label="🌐 Language",
                scale=1
            )
            title = gr.Markdown("# RAG Chat Demo")
        
        # Dynamic components
        upload = gr.File(label="Upload Documents")
        index_btn = gr.Button("Index Documents", variant="primary")
        msg = gr.Textbox(label="Message", placeholder="Type your message...")
        send_btn = gr.Button("Send", variant="primary")
        
        def update_language(language):
            t = TRANSLATIONS.get(language, TRANSLATIONS["en"])
            return (
                f"# {t['title']}",
                gr.update(label=t["upload"]),
                gr.update(value=t["index"]),
                gr.update(placeholder=t["message"]),
                gr.update(value=t["send"])
            )
        
        lang.change(
            update_language,
            [lang],
            [title, upload, index_btn, msg, send_btn]
        )
    
    return demo

print("Multi-language demo created!")
# multilang_demo = create_multilang_demo()
# multilang_demo.launch()

## Key Takeaways

1. **Gradio Blocks** provides full control over layout and styling
2. **Streaming** makes chat interfaces feel much more responsive
3. **Analytics** help you understand how users interact with your demo
4. **Export** functionality adds professional polish
5. **Internationalization** expands your demo's reach

These solutions demonstrate production-quality patterns you can adapt for your own projects.